In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import pandas as pd
from PyFunctions import Functions as func
from PyFunctions import var
from PyFunctions import Viz
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.applications import imagenet_utils
from keras.applications.vgg16 import VGG16
import cv2

# Mobilenet

In [ ]:
def get_mobilenet(dim):
    model = Sequential()
    optimizer = Adam(lr = .0005)
    baseModel = MobileNetV2(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim))
    
    
        
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(7, 7)))
    model.add(Flatten(name="flatten"))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation="softmax", name = 'Output'))

    
    for layer in baseModel.layers:
        layer.trainable = False
        
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model

### V1: Edge and Augmentation

In [ ]:
x_train, x_test, y_train, y_test = func.get_tts('mobilenet', version =1, edge = True, balance = True)


# cv2.imshow('test', x_train[25])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/MV1_Edge_Augment.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 16
    
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
mobilenet = get_mobilenet(dim =dim)

    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
mobilenet_history = mobilenet.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

In [ ]:
Viz.plot_loss_accuracy(mobilenet_history, 'dark_background', path = 'figures/MV1LossAcc_Edge_Augment.png')  

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV1_Edge_Augment.h5') #load the best weights before overfitting

Viz.plot_roc_auc(mobilenet, x_test, y_test, 'dark_background', 'figures/MV1ROC_Edge_Augment.png')

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV1_Edge_Augment.h5') #load the best weights before overfitting

y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'], theme = 'dark_background', 
                  path = 'figures/MV1CM_Edge_Augment.png')

### V1: Edge & No Augment

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/MV1_Edge_NoAugment.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 16
    
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
mobilenet = get_mobilenet(dim =dim)

    
mobilenet_history = mobilenet.fit(x_train, y_train, batch_size = batch_size,
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

In [ ]:
Viz.plot_loss_accuracy(mobilenet_history, 'dark_background', path = 'figures/MV1LossAcc_Edge_NoAugment.png')  

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV1_Edge_NoAugment.h5') #load the best weights before overfitting

Viz.plot_roc_auc(mobilenet, x_test, y_test, 'dark_background', 'figures/MV1ROC_Edge_NoAugment.png')

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV1_Edge_NoAugment.h5') #load the best weights before overfitting

y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'], theme = 'dark_background', 
                  path = 'figures/MV1CM_Edge_NoAugment.png')

### V1 No Edge & Augment

In [ ]:
x_train, x_test, y_train, y_test = func.get_tts('mobilenet', version =1, edge = False, balance = True)


# cv2.imshow('test', x_train[25])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/MV1_NoEdge_Augment.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 16
    
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
mobilenet = get_mobilenet(dim =dim)

    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
mobilenet_history = mobilenet.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

In [ ]:
Viz.plot_loss_accuracy(mobilenet_history, 'dark_background', path = 'figures/MV1LossAcc_NoEdge_Augment.png')  

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV1_NoEdge_Augment.h5') #load the best weights before overfitting

Viz.plot_roc_auc(mobilenet, x_test, y_test, 'dark_background', 'figures/MV1ROC_NoEdge_Augment.png')

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV1_NoEdge_Augment.h5') #load the best weights before overfitting

y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'], theme = 'dark_background', 
                  path = 'figures/MV1CM_NoEdge_Augment.png')

### V1: No Edge & No Augment

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/MV1_NoEdge_NoAugment.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 16
    
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
mobilenet = get_mobilenet(dim =dim)

    
mobilenet_history = mobilenet.fit(x_train, y_train, batch_size = batch_size,
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

In [ ]:
Viz.plot_loss_accuracy(mobilenet_history, 'dark_background', path = 'figures/MV1LossAcc_NoEdge_NoAugment.png')  

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV1_NoEdge_NoAugment.h5') #load the best weights before overfitting

Viz.plot_roc_auc(mobilenet, x_test, y_test, 'dark_background', 'figures/MV1ROC_NoEdge_NoAugment.png')

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV1_NoEdge_NoAugment.h5') #load the best weights before overfitting

y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'], theme = 'dark_background', 
                  path = 'figures/MV1CM_NoEdge_NoAugment.png')

### V2: Edge & Augmentation

In [ ]:
x_train, x_test, y_train, y_test = func.get_tts('mobilenet', version =2, edge = True, balance = True)

# cv2.imshow('test', x_train[25])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/MV2_Edge_Augment.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 16
    
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
mobilenet = get_mobilenet(dim =dim)

    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
mobilenet_history = mobilenet.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

In [ ]:
Viz.plot_loss_accuracy(mobilenet_history, 'dark_background', path = 'figures/MV2LossAcc_Edge_Augment.png')  

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV2_Edge_Augment.h5') #load the best weights before overfitting

Viz.plot_roc_auc(mobilenet, x_test, y_test, 'dark_background', 'figures/MV2ROC_Edge_Augment.png')

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV2_Edge_Augment.h5') #load the best weights before overfitting

y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'], theme = 'dark_background', 
                  path = 'figures/MV2CM_Edge_Augment.png')

### V2: Edge & No Augmentation

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/MV2_Edge_NoAugment.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 16
    
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
mobilenet = get_mobilenet(dim =dim)

    
mobilenet_history = mobilenet.fit(x_train, y_train, batch_size = batch_size,
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

In [ ]:
Viz.plot_loss_accuracy(mobilenet_history, 'dark_background', path = 'figures/MV2LossAcc_Edge_NoAugment.png')  

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV2_Edge_NoAugment.h5') #load the best weights before overfitting

Viz.plot_roc_auc(mobilenet, x_test, y_test, 'dark_background', 'figures/MV2ROC_Edge_NoAugment.png')

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV2_Edge_NoAugment.h5') #load the best weights before overfitting

y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'], theme = 'dark_background', 
                  path = 'figures/MV2CM_Edge_NoAugment.png')

### V2: No Edge & Augment

In [ ]:
x_train, x_test, y_train, y_test = func.get_tts('mobilenet', version =2, edge = False, balance = True)

# cv2.imshow('test', x_train[25])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/MV2_NoEdge_Augment.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 16
    
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
mobilenet = get_mobilenet(dim =dim)

    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
mobilenet_history = mobilenet.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

In [ ]:
Viz.plot_loss_accuracy(mobilenet_history, 'dark_background', path = 'figures/MV2LossAcc_NoEdge_Augment.png')  

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV2_NoEdge_Augment.h5') #load the best weights before overfitting

Viz.plot_roc_auc(mobilenet, x_test, y_test, 'dark_background', 'figures/MV2ROC_NoEdge_Augment.png')

In [ ]:
mobilenet = get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV2_NoEdge_Augment.h5') #load the best weights before overfitting

y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'], theme = 'dark_background', 
                  path = 'figures/MV2CM_NoEdge_Augment.png')

### V2: No Edge & No Augment